In [ ]:
%pip install --upgrade pip
%pip install torch
%pip install numpy
%pip install -U openai-whisper

In [ ]:
!mkdir -p model

In [ ]:
import numpy as np
import torch
import whisper

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"DEVICE that is used is {DEVICE}")

model = whisper.load_model("large")

print(
    f"Model is {'multilingual' if model.is_multilingual else 'English-only'} "
    f"and has {sum(np.prod(p.shape) for p in model.parameters()):,} parameters."
)

In [ ]:
torch.save(
    {
        'model_state_dict': model.state_dict(),
        'dims': model.dims.__dict__,
    },
    f'model/whisper.pt'
)

print("model saved")

In [ ]:
!cd model && tar -czvf model.tar.gz whisper.pt
!mv model/model.tar.gz .
!tar -tvf model.tar.gz

In [ ]:
import sagemaker


sess = sagemaker.session.Session()
bucket = sess.default_bucket()
prefix = f'whisper-deploy/'
s3_uri = f's3://{bucket}/{prefix}'
model_uri = sess.upload_data('model.tar.gz', bucket = bucket, key_prefix=f"{prefix}model")
print(f"{s3_uri}model/model.tar.gz")